In [1]:
import config.mongo_setup as mongo_setup
from models import Tweets, Replies
import pandas as pd
mongo_setup.global_init()

candidates = [('lhmandetta',2726301380),
('EduardoLeite_',123891446),
('DatenaOficial',68722955),
('SF_Moro',1113094855281000000),
('cirogomes',33374761),
('jdoriajr',64706049),
('LulaOficial',2670726740),
('jairbolsonaro',128372940),]

In [2]:

objs = Tweets.objects().all().values_list('id', 'tweet_id','author_id','created_at','text','lang','retweet_count','reply_count','like_count','quote_count')

corupus = pd.DataFrame.from_records(data=objs, columns=['id','tweet_id','author_id','created_at','text','lang','retweet_count','reply_count','like_count','quote_count'])
# corupus = corupus.drop(columns=['author_id','created_at','retweet_count','reply_count','like_count','quote_count','id'])
# corupus = corupus.rename(columns={"tweet_id": "id"})
corupus.head()

,id,tweet_id,author_id,created_at,text,lang,retweet_count,reply_count,like_count,quote_count
0,613ec5589eb2bd97646d86ad,1437158653623160837,2670726740,2021-09-12 20:58:18,@dilmabr 🙏🏼,und,123,76,6411,7
1,613ec5599eb2bd97646d86ae,1437054980884684804,2670726740,2021-09-12 14:06:21,Recomendação de leitura para o domingo: Amor à...,pt,1812,1529,18236,271
2,613ec5599eb2bd97646d86af,1436397090003881984,2670726740,2021-09-10 18:32:07,"“Olho pra trás, vejo a estrada que eu trilhei,...",pt,783,538,7294,84
3,613ec5599eb2bd97646d86b0,1436156966817181698,2670726740,2021-09-10 02:37:58,"@leandraleal Feliz Aniversário, companheira! 💫",pt,87,84,7371,7
4,613ec55a9eb2bd97646d86b1,1435999486162186244,2670726740,2021-09-09 16:12:11,@MussumAlive @manobrown 👊🏼,und,10,15,636,1


In [3]:
corupus = corupus.drop(columns=['author_id','created_at','retweet_count','reply_count','like_count','quote_count','id'])
corupus = corupus.rename(columns={"tweet_id": "id", 'lang':'language'})
corupus.head()

,id,text,language
0,1437158653623160837,@dilmabr 🙏🏼,und
1,1437054980884684804,Recomendação de leitura para o domingo: Amor à...,pt
2,1436397090003881984,"“Olho pra trás, vejo a estrada que eu trilhei,...",pt
3,1436156966817181698,"@leandraleal Feliz Aniversário, companheira! 💫",pt
4,1435999486162186244,@MussumAlive @manobrown 👊🏼,und


In [4]:
import json
js_file = corupus.to_json(orient="records")
json_file = json.loads(js_file)

In [8]:
from vcolors.colors import printS,printSBG, printW,printWBG , printF,printFBG


ImportError: cannot import name 'process_yaml' from partially initialized module 'main' (most likely due to a circular import) (c:\Users\victor.santos2\OneDrive - Grupo Marista\TCC\Twitter Sentiment Analysis\MongoTwitter\main.py)

reply 1 added to documents
reply 2 added to documents
reply 3 added to documents
reply 4 added to documents
reply 5 added to documents
reply 6 added to documents
reply 7 added to documents
reply 8 added to documents
reply 9 added to documents
reply 10 added to documents
[{'id': 1437254908391174151, 'text': '@paulo_a_andrade @jairbolsonaro 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂', 'language': 'pt'}, {'id': 1437254749422854152, 'text': '@fabiohpires @KatiaAMBarros @arnonco @jairbolsonaro Relaxa meu bom kkkk eu tenho a solução https://t.co/LbbuV2niCN', 'language': 'pt'}, {'id': 1437254632791699460, 'text': '@Antonio40867082 @jairbolsonaro @allanldsantos #MBLESTUPRADORES', 'language': 'pt'}, {'id': 1437254201105657862, 'text': '@luke_pallas @JosCamp61559152 @ClaudioMaduro @jairbolsonaro a amigao, quero nem saber, se apertar 13 trouxer preço mais barato nos supermercados eu to pouco me fudendo se é PT ou nao, o assalariado que tem familia pra cuidar n tem tempo pra fkr p